# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 6 2022 12:53PM,245748,21,626511,"NBTY Global, Inc.",Released
1,Sep 6 2022 12:51PM,245743,10,0085960253,ISDIN Corporation,Released
2,Sep 6 2022 12:51PM,245743,10,0085960252,ISDIN Corporation,Released
3,Sep 6 2022 12:51PM,245743,10,0085960263,ISDIN Corporation,Released
4,Sep 6 2022 12:51PM,245743,10,0085960264,ISDIN Corporation,Released
5,Sep 6 2022 12:51PM,245743,10,0085960267,ISDIN Corporation,Released
6,Sep 6 2022 12:51PM,245743,10,0085960269,ISDIN Corporation,Released
7,Sep 6 2022 12:51PM,245743,10,0085960265,ISDIN Corporation,Released
8,Sep 6 2022 12:51PM,245743,10,0085960266,ISDIN Corporation,Released
9,Sep 6 2022 12:51PM,245743,10,0085960270,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,245740,Released,10
28,245741,Released,1
29,245743,Released,61
30,245745,Released,1
31,245748,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
245740,NaN,10.0
245741,NaN,1.0
245743,NaN,61.0
245745,NaN,1.0
245748,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
245647,0.0,1.0
245654,1.0,0.0
245659,0.0,1.0
245689,0.0,7.0
245704,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
245647,0,1
245654,1,0
245659,0,1
245689,0,7
245704,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,245647,0,1
1,245654,1,0
2,245659,0,1
3,245689,0,7
4,245704,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,245647,,1
1,245654,1,
2,245659,,1
3,245689,,7
4,245704,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 6 2022 12:53PM,245748,21,"NBTY Global, Inc."
1,Sep 6 2022 12:51PM,245743,10,ISDIN Corporation
62,Sep 6 2022 12:48PM,245745,21,"NBTY Global, Inc."
63,Sep 6 2022 12:35PM,245740,10,ISDIN Corporation
73,Sep 6 2022 12:35PM,245741,19,"AdvaGen Pharma, Ltd"
74,Sep 6 2022 12:30PM,245739,19,"AdvaGen Pharma, Ltd"
102,Sep 6 2022 12:20PM,245736,10,Bio-PRF
103,Sep 6 2022 12:20PM,245735,10,Eywa Pharma Inc.
109,Sep 6 2022 12:18PM,245738,10,Ivaoes Animal Health
110,Sep 6 2022 12:17PM,245737,10,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 6 2022 12:53PM,245748,21,"NBTY Global, Inc.",,1
1,Sep 6 2022 12:51PM,245743,10,ISDIN Corporation,,61
2,Sep 6 2022 12:48PM,245745,21,"NBTY Global, Inc.",,1
3,Sep 6 2022 12:35PM,245740,10,ISDIN Corporation,,10
4,Sep 6 2022 12:35PM,245741,19,"AdvaGen Pharma, Ltd",,1
5,Sep 6 2022 12:30PM,245739,19,"AdvaGen Pharma, Ltd",2,26
6,Sep 6 2022 12:20PM,245736,10,Bio-PRF,,1
7,Sep 6 2022 12:20PM,245735,10,Eywa Pharma Inc.,,6
8,Sep 6 2022 12:18PM,245738,10,Ivaoes Animal Health,,1
9,Sep 6 2022 12:17PM,245737,10,"Nextsource Biotechnology, LLC",,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 12:53PM,245748,21,"NBTY Global, Inc.",1,
1,Sep 6 2022 12:51PM,245743,10,ISDIN Corporation,61,
2,Sep 6 2022 12:48PM,245745,21,"NBTY Global, Inc.",1,
3,Sep 6 2022 12:35PM,245740,10,ISDIN Corporation,10,
4,Sep 6 2022 12:35PM,245741,19,"AdvaGen Pharma, Ltd",1,
5,Sep 6 2022 12:30PM,245739,19,"AdvaGen Pharma, Ltd",26,2
6,Sep 6 2022 12:20PM,245736,10,Bio-PRF,1,
7,Sep 6 2022 12:20PM,245735,10,Eywa Pharma Inc.,6,
8,Sep 6 2022 12:18PM,245738,10,Ivaoes Animal Health,1,
9,Sep 6 2022 12:17PM,245737,10,"Nextsource Biotechnology, LLC",4,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 12:53PM,245748,21,"NBTY Global, Inc.",1,
1,Sep 6 2022 12:51PM,245743,10,ISDIN Corporation,61,
2,Sep 6 2022 12:48PM,245745,21,"NBTY Global, Inc.",1,
3,Sep 6 2022 12:35PM,245740,10,ISDIN Corporation,10,
4,Sep 6 2022 12:35PM,245741,19,"AdvaGen Pharma, Ltd",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 12:53PM,245748,21,"NBTY Global, Inc.",1.0,NaN
1,Sep 6 2022 12:51PM,245743,10,ISDIN Corporation,61.0,NaN
2,Sep 6 2022 12:48PM,245745,21,"NBTY Global, Inc.",1.0,NaN
3,Sep 6 2022 12:35PM,245740,10,ISDIN Corporation,10.0,NaN
4,Sep 6 2022 12:35PM,245741,19,"AdvaGen Pharma, Ltd",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 12:53PM,245748,21,"NBTY Global, Inc.",1.0,0.0
1,Sep 6 2022 12:51PM,245743,10,ISDIN Corporation,61.0,0.0
2,Sep 6 2022 12:48PM,245745,21,"NBTY Global, Inc.",1.0,0.0
3,Sep 6 2022 12:35PM,245740,10,ISDIN Corporation,10.0,0.0
4,Sep 6 2022 12:35PM,245741,19,"AdvaGen Pharma, Ltd",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2703016,235.0,34.0
12,245704,0.0,1.0
15,245713,6.0,36.0
16,491463,4.0,0.0
19,491480,27.0,2.0
20,737102,13.0,12.0
21,1719897,6.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2703016,235.0,34.0
1,12,245704,0.0,1.0
2,15,245713,6.0,36.0
3,16,491463,4.0,0.0
4,19,491480,27.0,2.0
5,20,737102,13.0,12.0
6,21,1719897,6.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,235.0,34.0
1,12,0.0,1.0
2,15,6.0,36.0
3,16,4.0,0.0
4,19,27.0,2.0
5,20,13.0,12.0
6,21,6.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,235.0
1,12,Released,0.0
2,15,Released,6.0
3,16,Released,4.0
4,19,Released,27.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,34.0,1.0,36.0,0.0,2.0,12.0,1.0
Released,235.0,0.0,6.0,4.0,27.0,13.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,34.0,1.0,36.0,0.0,2.0,12.0,1.0
1,Released,235.0,0.0,6.0,4.0,27.0,13.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,34.0,1.0,36.0,0.0,2.0,12.0,1.0
1,Released,235.0,0.0,6.0,4.0,27.0,13.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()